# Análisis Exploratorio de Datos (EDA)
## Logística, Inventario y Transacciones

En este notebook exploraremos tres conjuntos de datos relacionados con operaciones de logística y gestión de inventario.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import a_FundamentosCienciaDatos.Taller02.functions_eda as eda
import a_FundamentosCienciaDatos.Taller02.dictionaries as dict
warnings.filterwarnings('ignore')

# Configuración de visualización
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)

# Cargar los datos
transacciones = pd.read_csv('/Users/juangomez/Downloads/transacciones_logistica_v2.csv')
inventario = pd.read_csv('/Users/juangomez/Downloads/inventario_central_v2.csv')
feedback = pd.read_csv('/Users/juangomez/Downloads/feedback_clientes_v2.csv')

print("="*80)
print("DATOS CARGADOS EXITOSAMENTE")
print("="*80)
print(f"\nTransacciones: {transacciones.shape}")
print(f"Inventario: {inventario.shape}")
print(f"Feedback: {feedback.shape}")

DATOS CARGADOS EXITOSAMENTE

Transacciones: (10000, 10)
Inventario: (2500, 8)
Feedback: (4500, 9)


## 2. Análisis de Inventario

### 2.1 - Acercamiento inicial

In [ ]:
print("\n" + "="*80)
print("INFORMACIÓN DETALLADA - INVENTARIO")
print("="*80)
print(f"Inventario: {inventario.shape}")
print(inventario.info())
print("\n" + "="*80)
print("ESTADÍSTICAS DESCRIPTIVAS NUMÉRICAS - INVENTARIO")
print("="*80)
print(inventario.select_dtypes(include=['number']).describe())

### 2.2 Limpieza de inventarios

In [ ]:
# Ejecutar saneamiento y obtener reporte (devuelve: DataFrame saneado, DataFrame reporte)
inventario, inventarios_report = eda.sanitize_inventario(inventario)

print("\nResumen de procesos de saneamiento (Inventario):")
display(inventarios_report)

# Resumen adicional en formato tabla: absoluto y porcentaje respecto al total de filas del inventario
total_rows = inventario.shape[0]
summary = inventarios_report.copy()
summary['Porcentaje'] = (summary['Filas_afectadas'] / total_rows * 100).round(2)
summary = summary.sort_values('Filas_afectadas', ascending=False).reset_index(drop=True)

print("\nResumen (absoluto y % respecto al inventario):")
display(summary.style.format({'Porcentaje': '{:.2f}%'}))

print("\n" + "="*80)
print("SANITIZACIÓN DE DATOS COMPLETADA - INVENTARIO")
print("="*80)

print("\n" + "="*80)
print("INFORMACIÓN DETALLADA - INVENTARIO")
print("="*80)
print(f"Inventario: {inventario.shape}")
print(inventario.info())
print("\n" + "="*80)
print("ESTADÍSTICAS DESCRIPTIVAS NUMÉRICAS - INVENTARIO")
print("="*80)
print(inventario.select_dtypes(include=['number']).describe())
eda.print_table("PRIMERAS FILAS - INVENTARIO", inventario.head())

## 3. Análisis de Transacciones

### 3.1 - Acercamiento inicial

In [ ]:
print("\n" + "="*80)
print("INFORMACIÓN DETALLADA - TRANSACCIONES")
print("="*80)
print(f"Transacciones: {transacciones.shape}")
print(transacciones.info())
print("\n" + "="*80)
print("ESTADÍSTICAS DESCRIPTIVAS NUMÉRICAS - TRANSACCIONES")
print("="*80)
print(transacciones.select_dtypes(include=['number']).describe())

### 3.2 - Limpieza de Transacciones

In [ ]:
# Ejecutar saneamiento y obtener reporte (devuelve: DataFrame saneado, DataFrame reporte)
transacciones, transacciones_report = eda.sanitize_transacciones(transacciones)

print("\nResumen de procesos de saneamiento (Transacciones):")
display(transacciones_report)

# Resumen adicional en formato tabla: absoluto y porcentaje respecto al total de filas del inventario
total_rows = transacciones.shape[0]
summary = transacciones_report.copy()
summary['Porcentaje'] = (summary['Filas_afectadas'] / total_rows * 100).round(2)
summary = summary.sort_values('Filas_afectadas', ascending=False).reset_index(drop=True)

print("\nResumen (absoluto y % respecto al inventario):")
display(summary.style.format({'Porcentaje': '{:.2f}%'}))

print("\n" + "="*80)
print("SANITIZACIÓN DE DATOS COMPLETADA - TRANSACCIONES")
print("="*80)

print("\n" + "="*80)
print("INFORMACIÓN DETALLADA - TRANSACCIONES")
print("="*80)
print(f"Transacciones: {transacciones.shape}")
print(transacciones.info())
print("\n" + "="*80)
print("ESTADÍSTICAS DESCRIPTIVAS NUMÉRICAS - TRANSACCIONES")
print("="*80)
print(transacciones.select_dtypes(include=['number']).describe())
eda.print_table("PRIMERAS FILAS - TRANSACCIONES", transacciones.head())

In [ ]:
transacciones = eda.imputar_costo_envio_knn(transacciones, n_neighbors=5)

transacciones["Costo_Envio"].isna().sum()
transacciones["Costo_Envio_Imputado"].value_counts()

In [ ]:
transacciones = eda.enriquecer_con_estado_envio_reglas(
    transacciones=transacciones,
    inventario=inventario,
    hoy="2026-02-01",   # o la fecha de referencia de tu análisis
    margen_dias=2
)

transacciones["Estado_Envio_Reglas"].value_counts()

In [2]:
print("\n" + "="*80)
print("INFORMACIÓN DETALLADA - FEEDBACK")
print("="*80)
print(f"Feedback: {feedback.shape}")
print(feedback.info())
print("\n" + "="*80)
print("ESTADÍSTICAS DESCRIPTIVAS NUMÉRICAS - FEEDBACK")
print("="*80)
print(feedback.select_dtypes(include=['number']).describe())


INFORMACIÓN DETALLADA - FEEDBACK
Feedback: (4500, 9)
<class 'pandas.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Feedback_ID             4500 non-null   str    
 1   Transaccion_ID          4500 non-null   str    
 2   Rating_Producto         4500 non-null   int64  
 3   Rating_Logistica        4500 non-null   int64  
 4   Comentario_Texto        3843 non-null   str    
 5   Recomienda_Marca        3381 non-null   str    
 6   Ticket_Soporte_Abierto  4500 non-null   str    
 7   Edad_Cliente            4500 non-null   int64  
 8   Satisfaccion_NPS        4500 non-null   float64
dtypes: float64(1), int64(3), str(5)
memory usage: 316.5 KB
None

ESTADÍSTICAS DESCRIPTIVAS NUMÉRICAS - FEEDBACK
       Rating_Producto  Rating_Logistica  Edad_Cliente  Satisfaccion_NPS
count      4500.000000       4500.000000   4500.000000       4500.000000
mean     

In [7]:
feedback = limpiar_feedback_basico(feedback)

feedback["Satisfaccion_NPS_Grupo"].value_counts(normalize=True)


Satisfaccion_NPS_Grupo
muy_insatisfecho                   0.495333
satisfecho                         0.202667
neutro_o_ligeramente_satisfecho    0.155556
muy_satisfecho                     0.146444
Name: proportion, dtype: float64

In [ ]:
def validar_unicidad_feedback(feedback: pd.DataFrame) -> None:
    """Imprime chequeos de unicidad para Feedback_ID y Transaccion_ID.

    - Feedback_ID debería ser único.
    - Transaccion_ID puede repetirse (uno a muchos), pero revisamos cuántas repeticiones hay.
    - También busca filas completamente duplicadas.
    """
    print("Total filas:", len(feedback))

    # 1) Unicidad de Feedback_ID
    n_unique_feedback = feedback["Feedback_ID"].nunique()
    print("Feedback_ID únicos:", n_unique_feedback)
    if n_unique_feedback < len(feedback):
        print("--> Hay Feedback_ID duplicados")
        dup_feedback = feedback[feedback["Feedback_ID"].duplicated(keep=False)]
        print("Cantidad de filas con Feedback_ID duplicado:", len(dup_feedback))
    else:
        print("--> Feedback_ID es único")

    # 2) Repetición de Transaccion_ID
    n_unique_trans = feedback["Transaccion_ID"].nunique()
    print("Transaccion_ID únicos:", n_unique_trans)

    conteo_por_trans = feedback["Transaccion_ID"].value_counts()
    multi_feedback = conteo_por_trans[conteo_por_trans > 1]
    print("Transacciones con más de un feedback:", len(multi_feedback))

    if len(multi_feedback) > 0:
        ejemplo_id = multi_feedback.index[0]
        print("Ejemplo de Transaccion_ID con múltiples feedbacks:", ejemplo_id)
        print(feedback[feedback["Transaccion_ID"] == ejemplo_id].head())

    # 3) Filas completamente duplicadas (todas las columnas iguales)
    dup_exactos = feedback.duplicated(keep=False)
    print("Filas completamente duplicadas:", dup_exactos.sum())

validar_unicidad_feedback(feedback)

In [ ]:
def explorar_diccionarios_feedback(feedback: pd.DataFrame) -> None:
    """Muestra valores únicos y frecuencias básicas de columnas categóricas clave."""
    cols = ["Recomienda_Marca", "Ticket_Soporte_Abierto"]
    for col in cols:
        if col not in feedback.columns:
            print(f"Columna '{col}' no existe en el DataFrame.")
            continue

        print("\n==========", col, "=========")
        print("Valores únicos y conteos:")
        print(feedback[col].value_counts(dropna=False))

explorar_diccionarios_feedback(feedback)